In [ ]:
import weave
from weave.legacy.weave.ecosystem import lens

# Composite Histograms

Let's plot some histograms.

## Base case: One series, no customization

In [ ]:
lens.histogram([[1.0, 2, 3, -1.4]], 1.0, "Composite Histogram", ['series'])

## Two series with customization

In [ ]:
lens.histogram([[-1.8, -0.4, 0.1, -0.9, -0.77, 0.5, 0.7, 0.2, 0.1, 0.8],[0.9, 0.4, 0.5, 0.6, 1.2, 1.5, 1.6, 1.7, 2, 2.4, 2.3, 2.1]], 0.5, "My Histogram", ["series A", "series B"])

## Three series

In [ ]:
lens.histogram([[1.0, 2, 3, -1.4],[8.1, 2.2, 3.3, 5.6, 1.1], [1.6, 1.7, 1.8, -2.0, 5]], 0.5, 'Composite histogram', ['a', 'b', 'c'])

## More than three series :)

In [ ]:
lens.histogram([[1.0, -20.0, -25.0, -15.0, 2.0, 3.0, 2.0, 2.0, 2.0, 2.0, 4.0, 5.0, 6.0], [1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 3.0, 3.0, 3.0, 3.0, 2.0, 2.0, 2.0],[16.2, 17.9, 18.2, 2.3, -5.2, 6.0, 7.2, -10.9, 8.2,41.0], [1.3, 2.2, 5.3, 7.7, 8.8, 9.0, 2.0, 3.2, 43.7, 4.2, 5.1, 4.5, 6.9], [1.0, 2.1, 3.0, 4.1, 19.0, 6.0, 12.2, 14.9, 15.0, 16.1, 17.2]], 5.0, 'Composite histogram', ['a', 'b', 'c', 'd', 'e'])

In [ ]:
# weave-test-skip: in-notebook ops don't yet work with server
# super simple example for now to illustrate fetching W&B data & plotting histograms
from wandb.apis import public as wandb_api
from PIL import Image

@weave.op(render_info={"type": "function"})
def run_accuracy_barchart(runs: list[wandb_api.Run]) -> Image.Image:
    runs = list(runs) #[:100] # could shorten this list
    acc = [r.summary_metrics.get("acc") or 0.0 for r in runs]
    car_acc = [r.summary_metrics.get("car_acc") or 0.0 for r in runs]
    traffic_acc = [r.summary_metrics.get("traffic_acc") or 0.0 for r in runs]
    road_acc = [r.summary_metrics.get("road_acc") or 0.0 for r in runs]
    
    # filter out NaNs
    for l in [acc, car_acc, traffic_acc, road_acc]:
        if "NaN" in l:
            l = l.remove("NaN")

    return weave.use(lens.histogram([acc, car_acc, traffic_acc, road_acc],
                                    bin_size=0.05,
                                    chart_title="Semantic Segmentation Accuracy Across Experiments by Label Class", 
                                    series_names=["Overall acc", "Car acc", "Traffic acc", "Road acc"]
                                   )
                    )

runs = weave.legacy.weave.ops.project('stacey', 'deep-drive').runs()
run_accuracy_barchart(runs)